In [1]:
import sys 
sys.path.append('..')

import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from plotnine.data import diamonds

from apps.pyspark_tools import table_to_PandasDF_rdd,PandasDF_rdd_to_table

spark = (
    SparkSession
    .builder
    .appName('test')
    .config("spark.executor.cores","8")
    .config("spark.default.parallelism","600")
    .getOrCreate()
)
spark

22/05/07 13:35:59 WARN Utils: Your hostname, Lamian resolves to a loopback address: 127.0.1.1; using 172.29.242.201 instead (on interface eth0)
22/05/07 13:35:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/07 13:36:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
table = spark.createDataFrame(data=diamonds)
table.show()

+-----+---------+-----+-------+-----+-----+-----+----+----+----+
|carat|      cut|color|clarity|depth|table|price|   x|   y|   z|
+-----+---------+-----+-------+-----+-----+-----+----+----+----+
| 0.23|    Ideal|    E|    SI2| 61.5| 55.0|  326|3.95|3.98|2.43|
| 0.21|  Premium|    E|    SI1| 59.8| 61.0|  326|3.89|3.84|2.31|
| 0.23|     Good|    E|    VS1| 56.9| 65.0|  327|4.05|4.07|2.31|
| 0.29|  Premium|    I|    VS2| 62.4| 58.0|  334| 4.2|4.23|2.63|
| 0.31|     Good|    J|    SI2| 63.3| 58.0|  335|4.34|4.35|2.75|
| 0.24|Very Good|    J|   VVS2| 62.8| 57.0|  336|3.94|3.96|2.48|
| 0.24|Very Good|    I|   VVS1| 62.3| 57.0|  336|3.95|3.98|2.47|
| 0.26|Very Good|    H|    SI1| 61.9| 55.0|  337|4.07|4.11|2.53|
| 0.22|     Fair|    E|    VS2| 65.1| 61.0|  337|3.87|3.78|2.49|
| 0.23|Very Good|    H|    VS1| 59.4| 61.0|  338| 4.0|4.05|2.39|
|  0.3|     Good|    J|    SI1| 64.0| 55.0|  339|4.25|4.28|2.73|
| 0.23|    Ideal|    J|    VS1| 62.8| 56.0|  340|3.93| 3.9|2.46|
| 0.22|  Premium|    F|  

In [3]:
rdd = table_to_PandasDF_rdd(table=table,groupby_key=['color','clarity','cut'])
rdd2 = rdd.mapValues(lambda dt:dt.assign(t=lambda dt2:dt2.x+dt2.y+dt2.z).drop(['color','clarity','cut'],axis=1))

In [5]:
PandasDF_rdd_to_table(rdd2).dtypes

[('carat', 'double'),
 ('depth', 'double'),
 ('table', 'double'),
 ('price', 'double'),
 ('x', 'double'),
 ('y', 'double'),
 ('z', 'double'),
 ('t', 'double')]

In [10]:
x = list('abc')
y = list('efg')

t = zip(x,y)

list(zip(*t))

[('a', 'b', 'c'), ('e', 'f', 'g')]